https://towardsdatascience.com/work-with-geospatial-data-and-create-interactive-maps-using-geopy-and-plotly-28178d2868f1

In [1]:
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
import plotly.graph_objects as go
import numpy as np
from random import randint
import pandas as pd
import plotly
import plotly.express as px
from urllib.request import urlopen
from geopy.distance import distance
from geopy.geocoders import Nominatim
import plotly.offline as pyo


In [4]:
df = pd.read_csv("points-of-interest-google2.csv")
df.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,zip_code,primary_category
0,H&R Block,"H&R Block, 802 Lower Fayetteville Rd Ste C, Ne...",0x88f4c6329748b5b7:0x412418d65ff2fcb7,NaN,33.374876,-84.762020,"['Tax preparation service', 'Tax consultant']",4.8,58,NaN,"{'Thursday': '10AM–5PM', 'Friday': 'Closed', '...","{'Health & safety': ['Mask required', 'Staff w...",Open ⋅ Closes 5PM,"['0x88f4c6685cd3510f:0x154ee5bc850164a6', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,30263,Tax preparation service
1,Gwinnett Community Bank,"Gwinnett Community Bank, 2775 Buford Hwy, Dulu...",0x88f5a28359cb48cd:0xacc5f8fde9522f87,NaN,34.007883,-84.133413,"['Bank', 'ATM']",3.0,2,NaN,"{'Thursday': '9AM–4PM', 'Friday': '9AM–5PM', '...","{'Service options': ['Drive-through'], 'Access...",Open ⋅ Closes 4PM,"['0x88f599224d1b5ec5:0xcf6966bb9d99d4fc', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,30096,Bank
2,Corley Insurance Inc,"Corley Insurance Inc, 2855 Buford Hwy, Duluth,...",0x88f5a284e80502b3:0xfe92519aa603f4d9,NaN,34.006241,-84.136169,"['Insurance agency', 'Auto insurance agency', ...",4.1,8,NaN,"{'Thursday': '8:30AM–5:30PM', 'Friday': '8:30A...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 5:30PM,"['0x88f5a284fb58e8cd:0x8dd0cb71c88449ee', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,30096,Insurance agency
3,Kenn's Duluth Lock & Key,"Kenn's Duluth Lock & Key, 3395 Fox Street Suit...",0x88f5a262358cae1b:0x3039e83297ec5bed,NaN,33.998140,-84.144132,['Locksmith'],4.9,26,NaN,"{'Thursday': 'Open 24 hours', 'Friday': 'Open ...",NaN,Open 24 hours,"['0x88f5a2ebbbcebf19:0x6bdd4859a71bdb95', '0x8...",https://www.google.com/maps/place//data=!4m2!3...,30096,Locksmith
4,rangel services,"rangel services, 36 Reeves St, Norcross, GA 30071",0x88f5a112303230d1:0xde3124e294aba58f,NaN,33.936178,-84.202939,['Construction company'],4.4,6,NaN,"{'Thursday': '8AM–6PM', 'Friday': '8AM–6PM', '...","{'Service options': ['Online estimates', 'Onsi...",Open ⋅ Closes 6PM,NaN,https://www.google.com/maps/place//data=!4m2!3...,30071,Construction company


In [5]:


geolocator = Nominatim(user_agent="jason", timeout=10)

pyo.init_notebook_mode(connected=True)

mapbox_access_token = "REMOVED FOR GIT"

# define a function to create buffer around a point
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
def geodesic_point_buffer(lat, lon, miles):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)), 
        proj_wgs84)
    buf = Point(0, 0).buffer(miles * 1000/0.621371)  # distance in miles
    return transform(project, buf).exterior.coords[:]




# Defining center center point
target = geolocator.geocode('Atlanta, Georgia')
lat, lon = target.latitude, target.longitude

# filtering the dataframe to include markers only within the set distance
radius = 2  # miles
df['distance'] = df.apply(lambda row: distance((row['latitude'], row['longitude']), (lat, lon)).miles, axis=1)
df = df[df['distance'] <= radius]

# creating the map 
fig = go.Figure()
fig.add_trace(go.Scattermapbox(
        lat=df['latitude'],
        lon=df['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,
            color='blue'
        ),
        hoverinfo='text',  
        text=df['primary_category']  # This is the text to be displayed in the hovermode
    ))

# adding the buffer layer to the map
features = [{ "type": "Feature", "geometry": {"type": "LineString","coordinates": geodesic_point_buffer(lat, lon, radius)}}]
layers = [dict(
        sourcetype = 'geojson',
        source={"type": "FeatureCollection", 'features': features},           
        color= 'maroon',
        type = 'fill',   
        opacity=0.2,
        line=dict(width=1.5),
        below = "state-label-sm"
        )]

# Updating the layout of the map to display the primary category in the hovermode
fig.update_layout(
    title='POI Locations <br>Within {} Miles of Atlanta, Georgia'.format(radius),
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        layers=layers,
        bearing=0,
        center=dict(
            lat=lat,
            lon=lon
        ),
        pitch=0,
        zoom=10.5,  # Adjust the zoom level to better display the 3-mile radius
        style='light'
    ),
)

# Displaying the map
pyo.iplot(fig)


